In [1]:
# !pip install transformers peft matplotlib

In [2]:
import torch
print(torch.__version__)
torch.cuda.is_available()
# !pip uninstall torch -y
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

2.2.2


True

In [3]:
# !pip install transformers

import json
import math
import random
import torch


# train_data = MyLLMDataloader(4, tokenizer, "cleaned_TeleQnA_train_context_gte.json", shuffle=True)
import pandas as pd
import os

from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm

from string import Template
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import time
from statistics import mode
from transformers import logging
import os
import yaml
logging.set_verbosity_error()


In [4]:

config_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), '..')), "config/main.yaml")


with open(config_path, "r") as file:
    config = yaml.safe_load(file)
config["DATASET_PATH"] = config["PATHS"][config["DATASET_ID"]]

MODEL_ID = config["MODEL_ID"]
DATASET_ID = config["DATASET_ID"]
DATASET_PATH = config["DATASET_PATH"]


In [5]:

if  DATASET_ID == "medmcqa" and MODEL_ID in ["Qwen/Qwen2.5-3B-Instruct" ,"microsoft/phi-2", "google-t5/t5-3b"] :

    prompt_without_contex_train= Template('''Instruct = Youre a Medical Question Answering Expert, answer the following question. Please generate only answer choice (1, 2, 3 or 4)\n                                                                 
    $question
    $options
    Output: option ''')





elif DATASET_ID == "teleqna" and MODEL_ID in ["Qwen/Qwen2.5-3B-Instruct","google-t5/t5-3b"]  :
    
    prompt_without_contex_train= Template('''Instruct: $question
    Abbreviations: $abbreviation
            
    Considering the following contexts:
    context 1: $context1
    context 2: $context2
    context 3: $context3      
                                                                        
    $question
    $options
    Output: option ''')

elif DATASET_ID == "teleqna" and MODEL_ID == "microsoft/phi-2":


    prompt_without_contex_train= Template('''Instruct: $question
    Abbreviations: $abbreviation
            
    Considering the following contexts:
    context 1: $context1
    context 2: $context2
    context 3: $context3      
                                                                        
    $question
    $options
    Output: option ''')

elif DATASET_ID == "qasc" and MODEL_ID in ["Qwen/Qwen2.5-3B-Instruct","google-t5/t5-3b" ,"microsoft/phi-2"]:

    prompt_without_contex_train = Template('''Instruct: Answer the following question using the context provided, reason over it because only one of the context is relevant . Please generate only answer choice (1, 2, 3, 4, 5, 6, 7 or 8) without any explanations\n
    $question
    context: $context      
                                                            
    $options
    $question
    Output: option 
    ''')



In [6]:
def clean_question(question):
    for num in [14, 15, 16, 17, 18]:
        question = question.replace(f"[3GPP Release {num}]", "")
    return question

with open('time_no_bqkv'+DATASET_ID+"_"+ MODEL_ID.split('/')[1]+'.txt', 'w') as the_file:
    the_file.write('')

    # Load model directly
from transformers import AutoTokenizer, AutoModelWithLMHead

# tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-3b")

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
# torch.set_default_device("cuda")
if MODEL_ID == "google-t5/t5-3b":
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)
else:
    model = AutoModelForCausalLM.from_pretrained("/home/ubuntu/data/telcom_llm/medmcqa/logs_bias/medmcqa_qwen/model", trust_remote_code=True, device_map="auto", cache_dir = "/home/ubuntu/data/cache")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
class MyLLMDataloader:
    def __init__(self, batch_size, tokenizer, data, shuffle = False, val= False, k=20):
        ## initializations
        self.batch_size  = batch_size
        self.tokenizer  = tokenizer
        self.tokenizer.pad_token = self.tokenizer.eos_token
        # with open(data, "r") as f:
        #     self.data = json.load(f)
        if DATASET_ID == "medmcqa" or DATASET_ID == "qasc":
            self.data = []
            with open(data, "r") as f:
                test_data = f.readlines()

            for line in test_data:
                self.data.append(json.loads(line))
        elif DATASET_ID == "teleqna" :
            with open(data, "r") as f:
                self.data = json.load(f)
            self.all_examples = list(self.data.keys())

            
        # self.all_examples = list(self.data.keys())
        self.shuffle = shuffle
        self.val = val
        self.k = k
        self.n_data_points = math.ceil(len(self.data)/self.batch_size)
        self.indices = [i for i in range(self.n_data_points)]
        
    def __getitem__(self, idx):
        ## this gets a batch 

        option_header = ["option 1 ", "option 2 ", "option 3 ", "option 4 ", "option 5 "]
        batch_start_id = idx * self.batch_size
        mapper_ans = {"a":1, "b":2, "c":3, "d":4, "e":5}
        batch_end_id  = min(len(self.data), batch_start_id + self.batch_size) 
        batch = {"question_context":[], "answer":[]}

        batch_type =False
        
        if DATASET_ID == "medmcqa":
            for i in range(batch_start_id, batch_end_id):
                example = self.data[i]
                options = []
                opts = []
                for key in example.keys():
                    if key.startswith("op"):
                        if example[key] == None:
                            continue
                        options.append(example[key])
                        opts.append((example[key], mapper_ans[key.split("op")[1]] ))

        elif DATASET_ID == "teleqna":
            for i in range(batch_start_id, batch_end_id):
                example = self.data[self.all_examples[i]]
                options = []
                opts = []
                for key in example.keys():
                    if key.startswith("opt"):
                        if example[key] == None:
                            continue
                        options.append(example[key])
                        opts.append((example[key], key.split("option ")[1]))
            
        elif DATASET_ID == "qasc":
                mapper_ans = {"A":1, "B":2, "C":3, "D":4, "E":5, "F":6, "G":7, "H":8}
                option_header = ["option 1 ", "option 2 ", "option 3 ", "option 4 ", "option 5 ","option 6 ", "option 7 ", "option 8 " ]

                for i in range(batch_start_id, batch_end_id):
                    example = self.data[i]
                    options = []
                    opts = []
                    context = example["fact1"] + '\n' + example["fact2"]
                    for i, sample in enumerate(example['question']['choices']):
                        # print(key)
                        if example["answerKey"] == sample["label"]:
                            correct_option_id = mapper_ans[example["answerKey"]]
                            correct_option_txt = sample["text"]
                        
                            
                    
                        # options.append(option_header[i]+ sample['text'])
                        options.append(sample['text'])

                        opts.append((sample['text'], option_header[i].split("option")[1]))
                 
                explanation = example['combinedfact']

        string_opts = ' '.join(opt[0] for opt in opts)
        batch_prompts = []
        option_maps = []
        if not ("option" in string_opts or "above" in string_opts) and self.k>1:
                batch_type = True

                all_permutations = list(itertools.permutations(opts))
                all_permutations = random.sample(all_permutations, self.k if len(all_permutations)>self.k else len(all_permutations))
                for option_set in all_permutations:
                    option_map = []
                    options_with_header   = []
                    for z in range(len(option_set)):

                        options_with_header.append(option_header[z] +option_set[z][0])

                        
                        option_map.append(int(option_set[z][1]))
                    
                    
                    
                    # options_with_header = "\n".join(options_with_header)
                    if DATASET_ID == "medmcqa":                    
                        options_with_header = "\n".join(options_with_header)

                        prompt = prompt_without_contex_train.substitute(question = clean_question(example["question"]), options =options_with_header)
                    
                    elif DATASET_ID == "teleqna":
                        options_with_header = "\n".join(options_with_header)
                        prompt = prompt_without_contex_train.substitute(question = clean_question(example["question"]),\
                        abbreviation='\n'.join(example["abbreviation"]), context1 = '\n'.join(example["context_qwen2"][:2])[:512] , context2 = '\n'.join(example["context_gle"])[:512], context3 = '\n'.join(example["context_bm"][:2])[:512],
                        options =options_with_header)     
                    
                    elif DATASET_ID == "qasc":

                        options_with_header = "\n".join(options_with_header)

                        prompt = prompt_without_contex_train.substitute( question  = example['question']['stem'], context=  context, options = options_with_header)
                    # print(question_context, prompt)
                    batch_prompts.append(prompt)
                    option_maps.append(option_map)
                batch["question_context"] += batch_prompts


                    


        else:
            if DATASET_ID == "medmcqa":
            
                options_with_header = [option_header[i] +options[i] for i in range(len(options)) ]
            
                options_with_header = "\n".join(options_with_header)
                prompt = prompt_without_contex_train.substitute(question = clean_question(example["question"]), options =options_with_header)
            elif DATASET_ID == "teleqna":
                options_with_header = [option_header[i] +options[i] for i in range(len(options)) ]
                options_with_header = "\n".join(options_with_header)
                prompt = prompt_without_contex_train.substitute(question = clean_question(example["question"]),\
                abbreviation='\n'.join(example["abbreviation"]), context1 = '\n'.join(example["context_qwen2"][:2])[:512] , context2 = '\n'.join(example["context_gle"])[:512], context3 = '\n'.join(example["context_bm"][:2])[:512],
                options =options_with_header)

                # if MODEL_ID == "google-t5/t5-3b":
                #     prompt = prompt_without_contex_train.substitute(question = clean_question(example["question"]),\
                #     abbreviation='\n'.join(example["abbreviation"]), context1 = '\n'.join(example["context_qwen2"][:2])[:512] , context2 = '\n'.join(example["context_gle"])[:50], context3 = '\n'.join(example["context_bm"][:2])[:50],
                #     options =options_with_header)     
            
            elif DATASET_ID == "qasc" :
                correct_option_id = correct_option_id
                correct_option_txt_header = str(correct_option_id) + " " + correct_option_txt
                options_with_header = [option_header[i] +options[i] for i in range(len(options)) ]
                options_with_header = "\n".join(options_with_header)
                prompt = prompt_without_contex_train.substitute( question  = example['question']['stem'], context=  context, options = options_with_header)

                

            batch_prompts.append(prompt)
            

            


            batch["question_context"] += batch_prompts
            # batch["answer"] += [answer]

        self.tokenizer.padding_side = "left"
        q_tokens = self.tokenizer(batch["question_context"], padding="longest", return_tensors="pt")  
        self.tokenizer.padding_side = "right"
        # a_tokens = self.tokenizer(batch["answer"], padding="longest", return_tensors="pt")
        tokens = q_tokens
        attn_masks = q_tokens["attention_mask"]
        
       

        # attn_masks = torch.cat([q_tokens["attention_mask"], a_tokens["attention_mask"]], dim=1)
        # loss_mask = torch.cat([torch.zeros_like(q_tokens["attention_mask"]), a_tokens["attention_mask"]], dim=1)[:,1:]
   
        result = {
        "inp_ids":tokens["input_ids"],
        "inp_mask":attn_masks,## Causal Training
        "option_maps": option_maps
        }

        # result["loss_mask"] = loss_mask * result["out_mask"]
        # result["out_ids"][:,:q_tokens["input_ids"].size(1)-10] = self.tokenizer.eos_token_id

        return result       


            

    def __iter__(self):
        self.idx = 0
        return self

    def __next__(self):
        if self.idx >= self.n_data_points:
            self.idx = 0
            raise StopIteration
        temp_idx = self.indices[self.idx]
        self.idx += 1
        return self[temp_idx]
             





    def __len__(self):
        return self.n_data_points
    



        

In [9]:
k=20
testLoader = MyLLMDataloader(1, tokenizer, DATASET_PATH, val=True, shuffle=False, k=k)
for doc in testLoader:
    print(doc.keys())

    break

dict_keys(['inp_ids', 'inp_mask', 'option_maps'])


In [10]:
def forward_pass(model, batch):
    inp_ids = batch["inp_ids"].to(model.device)
    attn_mask = batch["inp_mask"].to(model.device)
    decoder_input_ids = tokenizer(["<pad>"], return_tensors="pt").input_ids

    if MODEL_ID != "google-t5/t5-3b":

        result = model(input_ids=inp_ids, attention_mask=attn_mask)
        logits = result.logits
        return logits
    else:
        result = model(
            input_ids=inp_ids,
            decoder_input_ids=decoder_input_ids,
            output_attentions=True,
            output_hidden_states=True
        )
        logits = result.logits


   
    return logits

In [11]:
def inference_with_bias(model, testLoader):
    """
    Perform inference on testLoader to compute biases in option permutations for LLMs.

    Args:
        model: The language model for inference.
        testLoader: An iterable containing test data. Each item is expected to have
                    the following keys:
                    - "inp_ids": Input IDs for the model.
                    - "inp_mask": Attention masks for the input IDs.
                    - "option_maps": A tensor mapping permutations of options, where
                      each row represents a specific permutation of option positions.
                      For example:
                      [[4, 1, 3, 2], [2, 1, 3, 4], [2, 4, 1, 3], ...]
                      This structure is used to evaluate model biases by reordering
                      the prediction probabilities to match these permutations.
    Returns:
        dict: A dictionary with "Answer_ID" and "Bias" for each example in testLoader.
    """
   

    # Initialize results dictionary
    my_ans = {"Answer_ID": [], "Bias": []}

    model.eval()  # Set model to evaluation mode
    if DATASET_ID != "qasc":
        option_ids = [tokenizer(o).input_ids[0] for o in ["1", "2", "3", "4", "5"]]
    else:
        option_ids = [tokenizer(o).input_ids[0] for o in ["1", "2", "3", "4", "5","6", "7", "8"]]


    pbar = tqdm(range(len(testLoader)), desc="Processing")

    for item in testLoader:
        if len(item["option_maps"]) > 0:
            # Process items with option maps
            with torch.inference_mode():
                with torch.autocast(device_type="cuda", dtype=torch.float16):
                    start = time.time()
                    logits = forward_pass(model, item)
                    # Log processing time
                    with open(f'time_bqkv{DATASET_ID}_{MODEL_ID.split("/")[1]}.txt', 'a') as the_file:
                        the_file.write(f"{time.time() - start}\n")

            # Compute probabilities and adjust with option_maps
            preds_probabs = torch.nn.functional.softmax(logits, dim=2)[:, -1, option_ids]
            preds_probabs = preds_probabs/preds_probabs.sum(dim=1).unsqueeze(1)

            z = torch.tensor(item["option_maps"]).cpu()
            reordered_probs = torch.gather(preds_probabs.cpu(), 1, torch.argsort(z, dim=1))

            # Calculate bias as the variance across permutations
            bias = torch.mean(torch.var(reordered_probs, dim=0))

            # Resolve final prediction based on the reordered probabilities
            preds = torch.mode(z.gather(1, preds_probabs.argmax(dim=1).cpu().unsqueeze(1)).squeeze(1)).values

            # Save results
            my_ans["Answer_ID"].append(preds.item())
            my_ans["Bias"].append(bias.item())
        else:
            continue
            # Handle cases without option maps
            with torch.inference_mode():
                with torch.autocast(device_type="cuda", dtype=torch.float16):
                    logits = forward_pass(model, item)
                preds = logits[:, -1, option_ids].argmax(dim=1) + 1
                my_ans["Answer_ID"].append(preds.item())
                

        # Save results to CSV and update progress bar
        pd.DataFrame(my_ans).to_csv("law_ans.csv", index=False)
        pbar.set_description(f"Prediction: {bias.item()}")
        pbar.update(1)

    return my_ans


In [12]:
k = 20
testLoader = MyLLMDataloader(1, tokenizer, DATASET_PATH, val=True, shuffle=False, k=k)

predictions = inference_with_bias(model, testLoader)

pd.DataFrame(predictions).to_csv(DATASET_ID+"_qwen3B_ans_k_"+ str(k)+".csv")

Processing:   0%|          | 0/4183 [00:00<?, ?it/s]

In [15]:
# labels = pd.read_csv("/home/ubuntu/data/llm_mcq_bias-main/experiments/qasc_qwen3B_ans_k_20.csv")
# print("Accuracy is ", sum(labels["Answer_ID"] == predictions["Answer_ID"])/len(labels["Answer_ID"] ))
print("Average Bias is = ", sum(predictions["Bias"])/len(predictions["Bias"]))

Average Bias is =  0.02959000940313199


In [14]:
sum(labels["Answer_ID"] != predictions["Answer_ID"])

NameError: name 'labels' is not defined